In [1]:
import numpy as np
import cv2 as cv
import glob

In [9]:
ground_truth = [] 
detected_points = [] 

points = []

point = np.array([0,0,0],dtype=np.float32)
for i in range(0,6):
    point[1] = 0
    for j in range(0,9):
        points.append([point[0],point[1],point[2]])
        point[1] += 21.5
    point[0] += 21.5 

points = np.array(points, dtype=np.float32)

images = glob.glob('Images/*.jpg')

test_img = cv.imread(images[0])
gray_test = cv.cvtColor(test_img, cv.COLOR_BGR2GRAY)

for i in images:
    img = cv.imread(i)
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

    ret, corners = cv.findChessboardCorners(gray, (9,6), cv.CALIB_CB_ADAPTIVE_THRESH + cv.CALIB_CB_FAST_CHECK + cv.CALIB_CB_NORMALIZE_IMAGE)

    if ret == True:
        ground_truth.append(points)
        corners2 = cv.cornerSubPix(gray,corners, (11,11), (-1,-1), (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001))
        detected_points.append(corners2)

        cv.drawChessboardCorners(img, (9,6), corners2, ret)
        cv.imshow('img', img)
        cv.waitKey(1000)

cv.destroyAllWindows()

In [10]:
ret, intrinsic_matrix, distortion, rotation, translation = cv.calibrateCamera(ground_truth, detected_points, gray_test.shape[::-1], None, None)


In [11]:
mean_error = 0
for i in range(len(ground_truth)):
    imgpoints2, _ = cv.projectPoints(ground_truth[i], rotation[i], translation[i], intrinsic_matrix, distortion)
    error = cv.norm(detected_points[i], imgpoints2, cv.NORM_L2)/len(imgpoints2)
    print("Reprojection Error of Image",i,"=",error,)
    mean_error += error
print("Mean Reprojection Error =",mean_error/13)

Reprojection Error of Image 0 = 0.07290379963579774
Reprojection Error of Image 1 = 0.08916218986685945
Reprojection Error of Image 2 = 0.1159750678564013
Reprojection Error of Image 3 = 0.13817273307939035
Reprojection Error of Image 4 = 0.062355126713086954
Reprojection Error of Image 5 = 0.07547120291245601
Reprojection Error of Image 6 = 0.03201701753001409
Reprojection Error of Image 7 = 0.05727789035017854
Reprojection Error of Image 8 = 0.07120680804111207
Reprojection Error of Image 9 = 0.07488903455136782
Reprojection Error of Image 10 = 0.10805784499400187
Reprojection Error of Image 11 = 0.12290643223159095
Reprojection Error of Image 12 = 0.11822729646816064
Mean Reprojection Error = 0.08758634186387827
